In [16]:
import numpy as np
import pandas as pd

In [17]:
pwd

u'/Users/stephane/classes/ds2/repos/gadat-project'

In [18]:
#1st step - read in a sample of the data
path = 'data/lc/'
url = path + 'LoanStats3a.csv'
df = pd.read_csv(url,skiprows=[0, 39788,39789,39790, 42540, 42541, 42542, 42543])


## Need some info to understand this mess

In [19]:
print df.shape
df[0:1]

(42535, 111)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [20]:
print "unique ids: {}".format(pd.value_counts(df.id).shape)
print "unique member_ids: {}".format(pd.value_counts(df.member_id).shape)

unique ids: (42535,)
unique member_ids: (42535,)


In [21]:
#loan_amnt doesn't always equal funded_amnt, but mostly does
adiff=df.loan_amnt - df.funded_amnt
print pd.value_counts(adiff).shape
pd.value_counts(adiff).head()

(519,)


0       40538
5000       16
7500       12
8000       11
3625       11
dtype: int64

In [22]:
colset1 = \
['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
'loan_status', 
 'out_prncp', 'out_prncp_inv',
 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
 
]

In [23]:
df=df[colset1]

In [24]:
df.shape

(42535, 18)

In [25]:
pd.value_counts(df.loan_status)

Fully Paid                                             32577
Charged Off                                             5605
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                 1522
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        47
In Grace Period                                           27
Late (16-30 days)                                          7
Default                                                    1
Name: loan_status, dtype: int64

In [26]:
pd.value_counts(df.grade)

B    12389
A    10183
C     8740
D     6016
E     3394
F     1301
G      512
Name: grade, dtype: int64

In [27]:
#should look at df.grade, and see if that predicts df.loan_status
#treat loan_status as binary: 0 Charged Off; 1 Fully Paid (ignore others)
#convert grade to numeric
#first look at plots
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [30]:
df2=df[(df.loan_status == 'Fully Paid') | (df.loan_status == 'Charged Off') ].copy()
# map loan_status to a numeric value 
df2['loan_status_num'] = df2.loan_status.map({'Charged Off':0, 'Fully Paid':1})
df2['grade_num'] = df2.grade.map({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6})

grouped=df2.groupby('grade')
probs=grouped.loan_status_num.sum()/grouped.loan_status_num.count()
#sns.lmplot(x='grade_num', y='loan_status_num', data=df2)

In [29]:
print probs

grade
A    0.939980
B    0.877099
C    0.826565
D    0.778331
E    0.727377
F    0.663839
G    0.656463
Name: loan_status_num, dtype: float64
